# Edge Analysis & Threshold Optimization

This notebook analyzes the relationship between **predicted edge** (Model Q75 - Implied Move) and **realized P&L**.

It uses data from `earnings_trades.db` (both executed trades and counterfactual non-trades) to answer:
1. Does higher predicted edge actually correlate with higher realized moves?
2. What is the optimal edge threshold for trading?
3. How does P&L change if we filter by edge?

**Data Source:** `data/earnings_trades.db`

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set style
plt.style.use('dark_background')
sns.set_palette("bright")

# Connect to DB
DB_PATH = '../data/earnings_trades.db'

def load_data():
    with sqlite3.connect(DB_PATH) as conn:
        # Load Trades (Executed)
        trades = pd.read_sql("""
            SELECT 
                ticker, earnings_date, earnings_timing, 
                'traded' as type,
                predicted_q75, implied_move, edge_q75,
                realized_move_pct, exit_pnl, exit_pnl_pct,
                entry_quoted_mid, entry_slippage
            FROM trades 
            WHERE status IN ('completed', 'exited', 'filled')
        """, conn)
        
        # Load Non-Trades (Counterfactuals)
        # Only those where we have realized move data
        non_trades = pd.read_sql("""
            SELECT 
                ticker, earnings_date, earnings_timing,
                'rejected' as type,
                predicted_q75, implied_move, predicted_edge as edge_q75,
                counterfactual_realized_move as realized_move_pct,
                counterfactual_pnl as exit_pnl, 
                NULL as exit_pnl_pct,
                straddle_premium/100 as entry_quoted_mid,
                0 as entry_slippage
            FROM non_trades
            WHERE counterfactual_realized_move IS NOT NULL
        """, conn)
        
    # Combine
    df = pd.concat([trades, non_trades], ignore_index=True)
    return df

df = load_data()
print(f"Loaded {len(df)} records ({len(df[df['type']=='traded'])} trades, {len(df[df['type']=='rejected'])} counterfactuals)")
df.head()

## 1. Win Rate vs Edge

Does the model correctly identify "beats"? 
- **True Win**: Realized Move > Implied Move
- **Edge**: Predicted Q75 > Implied Move

In [ ]:
# Drop records with missing critical data
df_clean = df.dropna(subset=['edge_q75', 'implied_move'])
print(f"Records with edge data: {len(df_clean)}")

if len(df_clean) > 0:
    # Did the stock beat the implied move?
    # For filled trades without realized move yet, we can't calculate win/loss
    df_clean = df_clean.dropna(subset=['realized_move_pct'])
    
    if len(df_clean) > 0:
        df_clean['win'] = df_clean['realized_move_pct'] > df_clean['implied_move']

        # Bin edge into buckets
        df_clean['edge_bucket'] = pd.cut(df_clean['edge_q75'], bins=[-0.2, 0, 0.05, 0.10, 0.15, 0.20, 1.0], 
                                        labels=['<0%', '0-5%', '5-10%', '10-15%', '15-20%', '>20%'])

        # Win rate by edge bucket
        win_rates = df_clean.groupby('edge_bucket')['win'].agg(['count', 'mean']).reset_index()
        win_rates.columns = ['Edge Bucket', 'Count', 'Win Rate']

        plt.figure(figsize=(10, 6))
        sns.barplot(data=win_rates, x='Edge Bucket', y='Win Rate', palette='viridis')
        plt.axhline(0.5, color='red', linestyle='--', label='50% Win Rate')
        plt.title('Win Rate (Realized > Implied) by Predicted Edge')
        plt.ylabel('Win Rate')
        plt.legend()
        plt.show()

        display(win_rates)
    else:
        print("No realized move data available yet.")
else:
    print("No records with edge predictions available.")

## 2. P&L Simulation by Threshold

If we only traded when `edge > X`, what would the total P&L be?

**Note:** P&L for non-trades is estimated using `counterfactual_pnl` (mid-to-mid, or with conservative spread assumption).

In [ ]:
if len(df_clean) > 0 and 'exit_pnl' in df_clean.columns and df_clean['exit_pnl'].notna().sum() > 0:
    thresholds = np.linspace(0, 0.20, 21)  # 0% to 20% edge
    results = []

    for thresh in thresholds:
        # Filter trades that meet the threshold
        subset = df_clean[df_clean['edge_q75'] >= thresh].copy()
        
        if len(subset) == 0:
            results.append({'threshold': thresh, 'total_pnl': 0, 'trade_count': 0, 'avg_pnl': 0, 'win_rate': 0})
            continue
        
        total_pnl = subset['exit_pnl'].sum()
        count = len(subset)
        avg_pnl = total_pnl / count
        win_rate = subset['win'].mean()
        
        results.append({
            'threshold': thresh,
            'total_pnl': total_pnl,
            'trade_count': count,
            'avg_pnl': avg_pnl,
            'win_rate': win_rate
        })

    res_df = pd.DataFrame(results)

    # Plot
    fig, ax1 = plt.subplots(figsize=(12, 6))

    ax1.set_xlabel('Edge Threshold (Predicted Q75 - Implied)')
    ax1.set_ylabel('Total P&L ($)', color='cyan')
    ax1.plot(res_df['threshold']*100, res_df['total_pnl'], color='cyan', marker='o', label='Total P&L')
    ax1.tick_params(axis='y', labelcolor='cyan')
    ax1.grid(True, alpha=0.2)

    ax2 = ax1.twinx()
    ax2.set_ylabel('Trade Count', color='white')
    ax2.bar(res_df['threshold']*100, res_df['trade_count'], color='white', alpha=0.3, width=0.8, label='Count')
    ax2.tick_params(axis='y', labelcolor='white')

    plt.title('Simulated P&L vs Edge Threshold')
    plt.show()

    display(res_df)
else:
    print("Not enough P&L data for simulation.")

## 3. Realized vs Implied Scatter

Visualizing the "Edge".

In [ ]:
if len(df_clean) > 0:
    plt.figure(figsize=(10, 10))

    # Scatter plot
    sns.scatterplot(data=df_clean, x='implied_move', y='realized_move_pct', 
                    hue='edge_q75', size='edge_q75', palette='coolwarm', sizes=(20, 200))

    # x=y line (Breakeven roughly)
    max_val = max(df_clean['implied_move'].max(), df_clean['realized_move_pct'].max())
    plt.plot([0, max_val], [0, max_val], 'r--', label='Realized = Implied')

    plt.xlabel('Implied Move')
    plt.ylabel('Realized Move')
    plt.title('Realized vs Implied Move (Color = Predicted Edge)')
    plt.legend()
    plt.axis('equal')
    plt.show()

## 4. Raw Data Inspection

In [ ]:
df_clean[['ticker', 'earnings_date', 'type', 'predicted_q75', 'implied_move', 'edge_q75', 'realized_move_pct', 'exit_pnl']]